In [119]:
# load data

import warnings
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
import numpy as np
from sklearn.utils import shuffle
mem = Memory('./mycache')

# The Memory class defines a context for lazy evaluation of function, by storing the results to the disk, 
# and not rerunning the function twice for the same arguments.
# It works by explicitly saving the output to a file and it is designed to work with non-hashable and potentially large input and output data types such as numpy arrays.
@mem.cache
def get_data(file_path):
    data = load_svmlight_file(file_path)
    return data

# classification task
# a1a
a1a_train = 'data/a1a/a1a.train'
a1a_test = 'data/a1a/a1a.test'
# a8a
a8a_train = 'data/a8a/a8a.train'
a8a_test = 'data/a8a/a8a.test'
#ijcnn
ijcnn_train = 'data/ijcnn/ijcnn.train'
ijcnn_test = 'data/ijcnn/ijcnn.test'

In [102]:
def fm(train_x, train_y, test_x, test_y, training_config):
    # pack parameters
    learning_rate = training_config['learning_rate'] if 'learning_rate' in training_config else 1e-2
    reg = training_config['reg'] if 'reg' in training_config else 1e-4
    factor = training_config['factor'] if 'factor' in training_config else 10
    task = training_config['task_type'] if 'task_type' in training_config else 'regression'
    
    if task not in ['classification', 'regression']:
        raise Exception('unsupported task type')
    
    epoch = training_config['epoch'] if 'epoch' in training_config else 10
    verbose = training_config['verbose'] if 'verbose' in training_config else True
    is_shuffle = training_config['shuffle'] if 'shuffle' in training_config else True
    
    seed = training_config['seed'] if 'seed' in training_config else 1
    np.random.seed(seed)
    
    training_num, p = train_x.shape
    test_num, p_test = test_x.shape
    
    if p != p_test:
        warnings.warn ('different dimensions in test and train data')
        p = max(p, p_test)
        
    # mandatory shuffle at the beginning
    x_train, y_train = shuffle(train_x, train_y, random_state=seed)
    
    train_loss_epochwise = []
    test_loss_epochwise = []
    
    for i in range(epoch):
        
        # initialization
        w0 = 0;
        W = np.zeros([1,p])
        V = 0.1*np.random.randn(p, factor)
        
        # if shuffle is true, do shuffle each epoch
        if is_shuffle:
            x_train, y_train = shuffle(train_x, train_y, random_state=seed)
        
        loss = 0
        for j in range(training_num):
            
            x = x_train[j]
            y = y_train[j]
            nz_idx = np.nonzero(x)[1]
            
            x = x[np.nonzero(x)]
            tmp = np.sum(np.multiply(np.matlib.repmat(x.T,1,factor),V[nz_idx,:]));
            factor_part = np.sum(np.square(tmp)) - np.sum(np.multiply(np.matlib.repmat(np.square(x.T),1,factor),np.square(V[nz_idx,:])))/2
            y_predict = (w0 + np.dot(W[:,nz_idx],x.T) + factor_part).item()
            
            if task == 'classification':
                err = 1/(1+np.exp(-y_predict*y))
                loss += (-np.log(err))
            
            if task == 'regression':
                err = y_predict - y
                loss += err**2
            
            # update
            if task == 'classification':
                w0_ = learning_rate * (err - 1) * y + 2 * reg * w0
                w0 -= w0_
                
                W_ = learning_rate * (err-1)*y*x + 2 * reg * W[:,nz_idx]
                W[:,nz_idx] -= W_
                
                V_ = learning_rate * (err - 1) * y *\
                np.multiply(
                    np.matlib.repmat(x.T,1,factor),(
                        np.matlib.repmat(np.dot(x,V[nz_idx,:]),len(nz_idx),1)-\
                        np.multiply(np.matlib.repmat(x.T,1,factor),V[nz_idx,:])
                    )
                ) + 2 * reg * V[nz_idx,:]
                V[nz_idx,:] -= V_
            if task == 'regression':
                w0_ = learning_rate * 2 * err + 2 * reg * w0
                w0 -= w0_
                
                W_ = learning_rate * 2 * err * x + 2 * reg * W[:,nz_idx]
                W[:,nz_idx] -= W_
                
                V_ = learning_rate * 2 * err *\
                np.multiply(
                    np.matlib.repmat(x.T,1,factor),(
                        np.matlib.repmat(np.dot(x,V[nz_idx,:]),len(nz_idx),1)-\
                        np.multiply(np.matlib.repmat(x.T,1,factor),V[nz_idx,:])
                    )
                ) + 2 * reg * V[nz_idx,:]
                V[nz_idx,:] -= V_
        
        train_loss_epochwise.append(loss/training_num)
        
        if verbose:
            print('epoch[%d]---train loss: %.4f\t' % (i,loss/training_num))
            
        loss = 0   
        # validation
        for j in range(test_num):
            x = test_x[j]
            y = test_y[j]
            nz_idx = np.nonzero(x)[1]
            
            x = x[np.nonzero(x)]
            tmp = np.sum(np.multiply(np.matlib.repmat(x.T,1,factor),V[nz_idx,:]));
            factor_part = np.sum(np.square(tmp)) - np.sum(np.multiply(np.matlib.repmat(np.square(x.T),1,factor),np.square(V[nz_idx,:])))/2
            y_predict = (w0 + np.dot(W[:,nz_idx],x.T) + factor_part).item()
            
            if task == 'classification':
                err = 1/(1+np.exp(-y_predict*y))
                loss += (-np.log(err))
            
            if task == 'regression':
                err = y_predict - y
                loss += err**2
                
        test_loss_epochwise.append(loss/test_num)
        
        if verbose:
            print('test loss: %.4f\n' % (loss/test_num))
        
    return train_loss_epochwise, test_loss_epochwise

In [121]:
train_data = ijcnn_train
test_data = ijcnn_test

train_x, train_y = get_data(train_data)
test_x, test_y = get_data(test_data)

training_config = {
    'learning_rate':1e-4,
    'reg':0,
    'task_type':'classification',
    'factor': 10,
    'verbose': True,
    'epoch': 10,
    'shuffle': False,
    'seed': 2017
}
fm(train_x, train_y, test_x, test_y, training_config);

________________________________________________________________________________
[Memory] Calling __main__-F%3A-Github-Robust-FM-__ipython-input__.get_data...
get_data('data/ijcnn/ijcnn.train')
_________________________________________________________get_data - 0.8s, 0.0min
________________________________________________________________________________
[Memory] Calling __main__-F%3A-Github-Robust-FM-__ipython-input__.get_data...
get_data('data/ijcnn/ijcnn.test')
_________________________________________________________get_data - 1.7s, 0.0min
epoch[0]---train loss: 0.5387	
test loss: 0.4220

epoch[1]---train loss: 0.5221	
test loss: 0.4168

epoch[2]---train loss: 0.5104	
test loss: 0.3952

epoch[3]---train loss: 0.5097	
test loss: 0.4001

epoch[4]---train loss: 0.5402	
test loss: 0.4114

epoch[5]---train loss: 0.5656	
test loss: 0.4501

epoch[6]---train loss: 0.5072	
test loss: 0.3980

epoch[7]---train loss: 0.5161	
test loss: 0.3998

epoch[8]---train loss: 0.5233	
test loss: 0.4194

e

([0.53874261197698359,
  0.52212617242787174,
  0.51035116216647214,
  0.50971187179941413,
  0.5402288711132357,
  0.56557451660732483,
  0.50719337145674137,
  0.51614400881132538,
  0.52330734136409451,
  0.51010196725309254],
 [0.42196492062644703,
  0.41681232939166735,
  0.39524568984388514,
  0.40014059938253754,
  0.41139868555174625,
  0.45005807247123458,
  0.39804388010211295,
  0.39984408229531448,
  0.41944901148156583,
  0.40426564870054948])